In [1]:
import numpy as np
import pandas as pd
import cv2
import os, glob
import nltk
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
import spacy
import tensorflow as tf

/home/mtnshh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mtnshh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mtnshh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mtnshh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
train = pd.read_csv('./dataset-flick8k/caption-token.csv', delimiter = '\t', header = None)
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40460 entries, 0 to 40459
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       40460 non-null  object
 1   1       40460 non-null  object
dtypes: object(2)
memory usage: 632.3+ KB


In [3]:
dictonary = set()
corpus = []


In [4]:

for i in range(0, len(train)):
    # print(train[1][i])
    text = train[1][i]
    # print(text)
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower()
    # nlp = spacy.load('en', disable=['parser', 'ner'])
    # doc = nlp(text)
    # text = " ".join([token.lemma_ for token in doc])
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text if not word in set(stopwords.words('english'))] 
    # text = []
    final = []
    for word in text:
        if word != "-PRON-":
            final.append(word)
    text = ' '.join(final)
    # print(text)
    # for word in text:
    #     dictonary.update(word)
    corpus.append(text)
    if i%100 ==  0:
        print(i)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [5]:
word_count = {}

for text in corpus:
    for w in  text.split(' '):
        word_count[w] = word_count.get(w, 0) + 1
    
vocab = [w for w in word_count if word_count[w] >= 10]

print(len(vocab))

1671


In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(vocab)
train_seqs = tokenizer.texts_to_sequences(vocab)

In [7]:
model = tf.keras.applications.InceptionV3(weights='imagenet')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
from PIL import Image
# from keras.models import Model
model_new = tf.keras.models.Model(model.input, model.layers[-2].output)

In [13]:
from time import time
from pickle import load, dump

def preprocess(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(299, 299))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.inception_v3.preprocess_input(x)
    return x

def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

start = time()
encoding_train = {}
cnt = 0
for img_path in glob.glob("./train/*"):
    img = img_path[8:]
    encoding_train[img] = encode(img_path)
    cnt = cnt + 1
    if cnt == 1:
        print(time()-start)
    if cnt%100 == 0:
        print(cnt)
        print("Time taken in seconds =", time()-start)
print("Time taken in seconds =", time()-start)

with open("./encoded_train_images.pkl", "wb") as encoded_pickle:
    dump(encoding_train, encoded_pickle)

0.1658036708831787
100
Time taken in seconds = 13.732467412948608
200
Time taken in seconds = 27.341927766799927
300
Time taken in seconds = 41.56098484992981
400
Time taken in seconds = 56.38016223907471
500
Time taken in seconds = 71.1683566570282
600
Time taken in seconds = 87.3818929195404
700
Time taken in seconds = 109.04961013793945
800
Time taken in seconds = 130.45548152923584
900
Time taken in seconds = 155.86932039260864
1000
Time taken in seconds = 180.62911176681519
1100
Time taken in seconds = 201.93630576133728
1200
Time taken in seconds = 224.61607313156128
1300
Time taken in seconds = 246.52128386497498
1400
Time taken in seconds = 267.8822412490845
1500
Time taken in seconds = 290.9380488395691
1600
Time taken in seconds = 312.74479961395264
1700
Time taken in seconds = 336.46473479270935
1800
Time taken in seconds = 359.0697350502014
1900
Time taken in seconds = 381.4822406768799
2000
Time taken in seconds = 402.7198836803436
2100
Time taken in seconds = 425.17145872

In [23]:
cnt = 0
for line in corpus:
#     line = line[17:len(line)-13]
    line = "startseq " + line + " endseq"
    corpus[cnt] = line
    cnt = cnt + 1

In [24]:
corpus[2]

'startseq little girl climbing wooden playhouse endseq'

In [25]:
vocab.append("startseq")
vocab.append("endseq")
print(len(vocab))

1673
